In [35]:
import pandas as pd
import numpy as np
import urllib.request, json 
import re
import pprint

In [36]:
pp = pprint.PrettyPrinter(depth=6)

In [37]:
modules = []
with urllib.request.urlopen("http://api.nusmods.com/2017-2018/1/moduleCodes.json") as url:
    modules = json.loads(url.read().decode())
    
modules = np.array(modules)


with urllib.request.urlopen("http://api.nusmods.com/2017-2018/2/moduleCodes.json") as url:
    modules_2 = json.loads(url.read().decode())
    
modules_2 = np.array(modules_2)

print(modules)
print(modules_2)
print('DSC5101' in modules)

['ACC1002' 'ACC1002X' 'ACC1006' ..., 'UTC3102' 'ZB3310' 'ZB3312']
['ACC1002' 'ACC1002X' 'ACC1006' ..., 'ZB3288' 'ZB3289' 'ZB4199']
True


In [38]:
# only take in the mods with these prefixes
# prefix = ["FIN", "DSC", "MNO", "MKT", "BSP", "ACC", "HR", "TR"]
modules_filtered = np.array(list(filter(lambda module: bool(re.compile("^(FIN|DSC|MNO|MKT|BSP|ACC|HR|TR)",).match(module)), modules)))

# remove "X" from suffix
modules_filtered = np.array(list(filter(lambda module: module[-1] != 'X', modules_filtered)))

modules_filtered_2 = np.array(list(filter(lambda module: bool(re.compile("^(FIN|DSC|MNO|MKT|BSP|ACC|HR|TR)",).match(module)), modules_2)))

# remove "X" from suffix
modules_filtered_2 = np.array(list(filter(lambda module: module[-1] != 'X', modules_filtered_2)))
modules_filtered_2
print(len(modules_filtered))

151


In [39]:
# define a function that gives us the json from the thing

def get_json(module_code):
    with urllib.request.urlopen("http://api.nusmods.com/2017-2018/1/modules/" + module_code + ".json") as url:
        data = json.loads(url.read().decode())
        
        # remove CorsBiddingStats
        try:
            del data['CorsBiddingStats']
        except:
            pass
        return data
    
    
def get_json_2(module_code):
    with urllib.request.urlopen("http://api.nusmods.com/2017-2018/2/modules/" + module_code + ".json") as url:
        data = json.loads(url.read().decode())
        
        # remove CorsBiddingStats
        try:
            del data['CorsBiddingStats']
        except:
            pass
        return data

pp.pprint((get_json('FIN2004')))
# pp.pprint(get_json('DSC2004'))

{'Department': 'Finance',
 'ExamDate': '2017-11-27T17:00+0800',
 'LecturePeriods': ['Tuesday Afternoon',
                    'Monday Morning',
                    'Monday Afternoon',
                    'Tuesday Morning',
                    'Wednesday Morning',
                    'Thursday Morning',
                    'Wednesday Afternoon',
                    'Friday Morning',
                    'Thursday Afternoon',
                    'Friday Afternoon'],
 'ModuleCode': 'FIN2004',
 'ModuleCredit': '4',
 'ModuleDescription': 'This course helps students to understand the key '
                      'concepts and tools in Finance. It provides a broad '
                      'overview of the financial environment under which a '
                      'firm operates. It equips the students with the '
                      'conceptual and analytical skills necessary to make '
                      'sound financial decisions for a firm. Topics to be '
                      'covered inc

In [40]:
headers = []
timeslots = ['0800', '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200']
days = ['mon', 'tue', 'wed', 'thu', 'fri']

# module columns
headers.extend(['module', 'name', 'sectional', 'module_sectional'])

# timeslots
for day in days:
    for timeslot in timeslots:
        headers.extend([day + timeslot])
        
def build_df(data):
    return pd.DataFrame(data, columns=headers)

x = []
for i in range(0, 79):
    x.append(i)
    
    
build_df([x])

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78


In [41]:
def convert_time_info_into_list(day, start, end):
    day = day.lower()[:3]
    started = False
    ended = False
    
    timelist = []
    
    for currday in days:
        for slot in timeslots:
            if currday+slot == day+start:
                started = True
            if currday+slot == day+end:
                ended = True
            if (started & ~ended):
                timelist.append(1)
            else:
                timelist.append(0)

    return timelist

convert_time_info_into_list('Monday', '1000', '1200')


dsc3214 = ['DSC3214', 'Optimisation', 'A', 'DSC3214_A']
dsc3214.extend(convert_time_info_into_list('Friday', '1400', '1700'))
build_df([dsc3214])
    

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,DSC3214,Optimisation,A,DSC3214_A,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [42]:
def convert_mod_info_into_lists(json):
    # this will create a LIST OF LIST, where each inner list is 1 module_sectional!!
    # [modulecode, sectional, mod_sectional, name, [day, start, end]]
    module = json['ModuleCode']
    title = json['ModuleTitle']
    try:
        timeslots = json['Timetable']
    except:
        print('Timetable not found for', module)
        blankslot = {}
        blankslot['ClassNo'] = ''
        blankslot['StartTime'] = '0'
        blankslot['EndTime'] = '0'
        blankslot['DayText'] = '0'
        timeslots = [blankslot]
    
    mylist = []
    
    for slot in timeslots:
        sect = slot['ClassNo']
        start = slot['StartTime']
        end = slot['EndTime']
        day = slot['DayText']
        mylist.append([[module, title, sect, module+sect], (day, start, end)])
    
    return mylist

fin2004 = convert_mod_info_into_lists(get_json('ACC1701'))
fin2004

[[['ACC1701', 'Accounting for Decision Makers', 'V1', 'ACC1701V1'],
  ('Thursday', '0900', '1100')],
 [['ACC1701', 'Accounting for Decision Makers', 'V2', 'ACC1701V2'],
  ('Wednesday', '0900', '1100')],
 [['ACC1701', 'Accounting for Decision Makers', 'V3', 'ACC1701V3'],
  ('Wednesday', '1200', '1400')],
 [['ACC1701', 'Accounting for Decision Makers', 'V01', 'ACC1701V01'],
  ('Monday', '0900', '1000')],
 [['ACC1701', 'Accounting for Decision Makers', 'V02', 'ACC1701V02'],
  ('Monday', '1000', '1100')],
 [['ACC1701', 'Accounting for Decision Makers', 'V03', 'ACC1701V03'],
  ('Monday', '1200', '1300')],
 [['ACC1701', 'Accounting for Decision Makers', 'V04', 'ACC1701V04'],
  ('Monday', '1300', '1400')],
 [['ACC1701', 'Accounting for Decision Makers', 'V05', 'ACC1701V05'],
  ('Friday', '0900', '1000')],
 [['ACC1701', 'Accounting for Decision Makers', 'V06', 'ACC1701V06'],
  ('Friday', '1000', '1100')],
 [['ACC1701', 'Accounting for Decision Makers', 'V07', 'ACC1701V07'],
  ('Friday', '1200'

In [43]:
lect_tut = [
    'ACC1002',
    'ACC1701',
    'BSP1005',
    'BSP1703',
    'BSP2001',
    'DSC2006',
    'DSC2008',
    'MNO2009'
]

In [44]:
def convert_modlist_into_df(modlist):
    newlist = []
    for mod in modlist:
#         print((mod[0][0] in lect_tut) & (len(mod[0][2]) == 3))
        if ((mod[0][0] in lect_tut) & (len(mod[0][2]) == 3)):
            mod_time = convert_time_info_into_list('mon', '0800', '9999') # fill up all the slots
            print('timetable building skipped for', mod[0][0], mod[0][2])
        else:
            mod_time = convert_time_info_into_list(mod[1][0], mod[1][1], mod[1][2])
        newmod = []
        newmod.extend(mod[0])
        newmod.extend(mod_time)
        newlist.append(newmod)
        
    return newlist

build_df(convert_modlist_into_df(fin2004))

timetable building skipped for ACC1701 V01
timetable building skipped for ACC1701 V02
timetable building skipped for ACC1701 V03
timetable building skipped for ACC1701 V04
timetable building skipped for ACC1701 V05
timetable building skipped for ACC1701 V06
timetable building skipped for ACC1701 V07
timetable building skipped for ACC1701 V08
timetable building skipped for ACC1701 V09
timetable building skipped for ACC1701 V10
timetable building skipped for ACC1701 V11
timetable building skipped for ACC1701 V12
timetable building skipped for ACC1701 V13
timetable building skipped for ACC1701 V14


,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,ACC1701,Accounting for Decision Makers,V1,ACC1701V1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACC1701,Accounting for Decision Makers,V2,ACC1701V2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACC1701,Accounting for Decision Makers,V3,ACC1701V3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACC1701,Accounting for Decision Makers,V01,ACC1701V01,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,ACC1701,Accounting for Decision Makers,V02,ACC1701V02,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,ACC1701,Accounting for Decision Makers,V03,ACC1701V03,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,ACC1701,Accounting for Decision Makers,V04,ACC1701V04,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,ACC1701,Accounting for Decision Makers,V05,ACC1701V05,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,ACC1701,Accounting for Decision Makers,V06,ACC1701V06,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,ACC1701,Accounting for Decision Makers,V07,ACC1701V07,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## Generating a list of mod_sectional schedules

First, to find a list of non-sectional mods (lectural-tutorial system) cos we won't be looking at those

In [45]:
mod_sect_list = []
for mod in modules_filtered:
    mod_json = get_json(mod)
    mod_sect_list.extend(convert_mod_info_into_lists(mod_json))
    
df = build_df(convert_modlist_into_df(mod_sect_list))

Timetable not found for ACC4619
Timetable not found for ACC4629
Timetable not found for DSC3229
Timetable not found for DSC4219
Timetable not found for FIN3129
Timetable not found for FIN4119
Timetable not found for FIN4129
Timetable not found for MKT3429
Timetable not found for MKT4419
Timetable not found for MKT4429
Timetable not found for MNO3329
Timetable not found for MNO4319
Timetable not found for TR3002N
Timetable not found for TR3202
Timetable not found for TR3203E
Timetable not found for TR3203N
Timetable not found for TR3203P
Timetable not found for TR3203T
Timetable not found for DSC3222M
Timetable not found for DSC3239
Timetable not found for DSC4229
Timetable not found for FIN3139
Timetable not found for MKT3439
Timetable not found for MNO4329
timetable building skipped for ACC1002 V01
timetable building skipped for ACC1002 V02
timetable building skipped for ACC1002 V03
timetable building skipped for ACC1002 V04
timetable building skipped for ACC1002 V05
timetable buildin

In [46]:
df.to_csv('1718_s1_modsect_timeslots.csv')

In [47]:
df_1 = df.copy()
df

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,ACC1002,Financial Accounting,V1,ACC1002V1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACC1002,Financial Accounting,V2,ACC1002V2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACC1002,Financial Accounting,V3,ACC1002V3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACC1002,Financial Accounting,V01,ACC1002V01,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,ACC1002,Financial Accounting,V02,ACC1002V02,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,ACC1002,Financial Accounting,V03,ACC1002V03,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,ACC1002,Financial Accounting,V04,ACC1002V04,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,ACC1002,Financial Accounting,V05,ACC1002V05,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,ACC1002,Financial Accounting,V06,ACC1002V06,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,ACC1002,Financial Accounting,V07,ACC1002V07,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [48]:
df_1['module'] = df_1['module'].apply(lambda x: x[:-1] if x[-1].isalpha() else x)
df_1

,module,name,sectional,module_sectional,mon0800,mon0900,mon1000,mon1100,mon1200,mon1300,...,fri1300,fri1400,fri1500,fri1600,fri1700,fri1800,fri1900,fri2000,fri2100,fri2200
0,ACC1002,Financial Accounting,V1,ACC1002V1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACC1002,Financial Accounting,V2,ACC1002V2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACC1002,Financial Accounting,V3,ACC1002V3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACC1002,Financial Accounting,V01,ACC1002V01,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,ACC1002,Financial Accounting,V02,ACC1002V02,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,ACC1002,Financial Accounting,V03,ACC1002V03,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,ACC1002,Financial Accounting,V04,ACC1002V04,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,ACC1002,Financial Accounting,V05,ACC1002V05,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,ACC1002,Financial Accounting,V06,ACC1002V06,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,ACC1002,Financial Accounting,V07,ACC1002V07,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [49]:
len(list(set(df_1['module'].tolist())))

121

In [50]:
df_1.to_csv('1718_s1_modsect_timeslots.csv')

In [51]:
mod_sect_list_2 = []
for mod in modules_filtered_2:
    mod_json = get_json_2(mod)
    mod_sect_list.extend(convert_mod_info_into_lists(mod_json))
    
df = build_df(convert_modlist_into_df(mod_sect_list_2))

Timetable not found for ACC1002
Timetable not found for ACC1006
Timetable not found for ACC1701
Timetable not found for ACC2002
Timetable not found for ACC3601
Timetable not found for ACC3603
Timetable not found for ACC3604
Timetable not found for ACC3605
Timetable not found for ACC3606
Timetable not found for ACC3613
Timetable not found for ACC3614
Timetable not found for ACC3616
Timetable not found for ACC3619
Timetable not found for ACC4611
Timetable not found for ACC4614
Timetable not found for ACC5001
Timetable not found for BSP1004
Timetable not found for BSP1005
Timetable not found for BSP2001
Timetable not found for BSP2005
Timetable not found for BSP3001A
Timetable not found for BSP3001B
Timetable not found for BSP3001C
Timetable not found for BSP3516
Timetable not found for DSC1007
Timetable not found for DSC2006
Timetable not found for DSC2008
Timetable not found for DSC3201
Timetable not found for DSC3202
Timetable not found for DSC3203
Timetable not found for DSC3214
Timet

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [52]:
df.to_csv('1718_s2_modsect_timeslots.csv')
# lol nothing at all XD

<br /><br /><br /><br /><br />
## Generating a Prereq/Preclu list

In [51]:
def convert_mod_into_prereq_preclu_list(json):
    # this will create a LIST OF LIST, where each inner list is 1 module_sectional!!
    # [modulecode, sectional, mod_sectional, name, [day, start, end]]
    module = json['ModuleCode']
    title = json['ModuleTitle']
    try:
        prereq = json['Prerequisite']
    except:
        prereq = ''
    try:
        preclu = json['Preclusion']
    except:
        preclu = ''
    
    return [module, title, prereq, preclu]

fin2004 = convert_mod_into_prereq_preclu_list(get_json('FIN2004'))
fin2004

['FIN2004',
 'Finance',
 'Students must have completed BK1003 or BZ1002 or BH1002 or FNA1002/ACC1002 or FNA1002X/ACC1002X or FNA1002E or BH1002E or EC3212 or EG1422 before they are allowed to take FIN2004.',
 'Students who have taken CS2251 or EC3209 or EC3333 or BK2004 or BZ2004 or BH2004 or FNA2004 are not allowed to take FIN2004. 1st Year BSc(PFM), all BSc (RE) and Computational Finance are not allowed to take FIN2004.']

In [52]:
prereq_preclu_list = []
for mod in modules_filtered:
    try:
        myjson = get_json(mod)
    except:
        myjson = {}
        myjson['ModuleCode'] = mod
        myjson['ModuleTitle'] = 'NOT FOUND'
    prereq_preclu_list.append(convert_mod_into_prereq_preclu_list(myjson))
    
prereq_df = pd.DataFrame(prereq_preclu_list, columns=['Module', 'Title', 'Prereq', 'Preclu'])
prereq_df

,Module,Title,Prereq,Preclu
0,ACC1002,Financial Accounting,,Students who have passed FNA1002 are not allow...
1,ACC1006,Accounting Information Systems,FNA1002 or ACC1002,Students who have passed FNA1006 are not allow...
2,ACC1701,Accounting for Decision Makers,,ACC1002; ACC1002X
3,ACC2002,Managerial Accounting,Students must have completed BK1003 or BZ1002 ...,BH2002 or BZ3102 or BK2001 or FNA2002 or IE4242
4,ACC3601,Corporate Accounting and Reporting,FNA1002 or ACC1002,BH3111 or BZ3101 or BK3106 or FNA3111
5,ACC3602,Managerial Planning and Control,FNA2002 or ACC2002,Students who have passed FNA3112 are not allow...
6,ACC3603,Assurance and Attestation,FNA1002/ACC1002 and FNA2002 (Students who are ...,Students who have passed FNA3121 are not allow...
7,ACC3604,Corporate and Securities Law,BSP1004,Students who have passed FNA3122 or LL4055 are...
8,ACC3605,Taxation,FNA1002/ACC1002 and BSP1004,Students who have passed FNA3127 or LL4056 are...
9,ACC3606,Advanced Corporate Accounting and Reporting,FNA3111 or ACC3601,Students who have passed FNA3123 are not allow...


In [53]:
prereq_df.to_csv('prereq_preclu.csv')

In [54]:
prereq_preclu_list = []
for mod in modules_filtered_2:
    try:
        myjson = get_json_2(mod)
    except:
        myjson = {}
        myjson['ModuleCode'] = mod
        myjson['ModuleTitle'] = 'NOT FOUND'
    prereq_preclu_list.append(convert_mod_into_prereq_preclu_list(myjson))
    
prereq_df = pd.DataFrame(prereq_preclu_list, columns=['Module', 'Title', 'Prereq', 'Preclu'])
prereq_df

,Module,Title,Prereq,Preclu
0,ACC1002,Financial Accounting,,Students who have passed FNA1002 are not allow...
1,ACC1006,Accounting Information Systems,FNA1002 or ACC1002,Students who have passed FNA1006 are not allow...
2,ACC1701,Accounting for Decision Makers,,ACC1002; ACC1002X
3,ACC2002,Managerial Accounting,Students must have completed BK1003 or BZ1002 ...,BH2002 or BZ3102 or BK2001 or FNA2002 or IE4242
4,ACC3601,Corporate Accounting and Reporting,FNA1002 or ACC1002,BH3111 or BZ3101 or BK3106 or FNA3111
5,ACC3603,Assurance and Attestation,FNA1002/ACC1002 and FNA2002 (Students who are ...,Students who have passed FNA3121 are not allow...
6,ACC3604,Corporate and Securities Law,BSP1004,Students who have passed FNA3122 or LL4055 are...
7,ACC3605,Taxation,FNA1002/ACC1002 and BSP1004,Students who have passed FNA3127 or LL4056 are...
8,ACC3606,Advanced Corporate Accounting and Reporting,FNA3111 or ACC3601,Students who have passed FNA3123 are not allow...
9,ACC3613,Advanced Assurance and Attestation,FNA3121 or ACC3603,Students who have passed FNA3128 are not allow...


In [55]:
prereq_df.to_csv('prereq_preclu_2.csv')

## Generating a list of mods belonging to s1/s2

In [69]:
modlist = sorted(list(set(df_1['module'].tolist())), key=str.lower)
for i in range(0, len(modlist)):
    print(i+2, modlist[i])

2 ACC1002
3 ACC1006
4 ACC1701
5 ACC2002
6 ACC3601
7 ACC3602
8 ACC3603
9 ACC3604
10 ACC3605
11 ACC3606
12 ACC3611
13 ACC3612
14 ACC3613
15 ACC3614
16 ACC3616
17 ACC3619
18 ACC4611
19 ACC4612
20 ACC4613
21 ACC4615
22 ACC4619
23 ACC4629
24 ACC5001
25 BSP1004
26 BSP1005
27 BSP1702
28 BSP1703
29 BSP2001
30 BSP2005
31 BSP3001
32 BSP3513
33 BSP3516
34 BSP4513
35 DSC1007
36 DSC2006
37 DSC2008
38 DSC3201
39 DSC3202
40 DSC3203
41 DSC3214
42 DSC3215
43 DSC3216
44 DSC3222
45 DSC3229
46 DSC3239
47 DSC4213
48 DSC4216
49 DSC4219
50 DSC4229
51 DSC5101
52 DSC5102
53 DSC5103
54 DSC5211
55 FIN2004
56 FIN3101
57 FIN3102
58 FIN3103
59 FIN3113
60 FIN3115
61 FIN3116
62 FIN3118
63 FIN3119
64 FIN3120
65 FIN3129
66 FIN3130
67 FIN3131
68 FIN3132
69 FIN3139
70 FIN4111
71 FIN4112
72 FIN4113
73 FIN4115
74 FIN4116
75 FIN4118
76 FIN4119
77 FIN4122
78 FIN4123
79 FIN4129
80 FIN6004
81 HR2002
82 MKT1003
83 MKT1705
84 MKT2401
85 MKT2411
86 MKT2412
87 MKT2414
88 MKT3402
89 MKT3412
90 MKT3415
91 MKT3418
92 MKT3420
93 MKT34